In [9]:
import pyfcctab as fcc
from intervaltree import intervaltree

In [2]:
tables = fcc.load()

In [12]:
help (tables.indices["R1"].tree.slice)

Help on method slice in module intervaltree.intervaltree:

slice(point, datafunc=None) method of intervaltree.intervaltree.IntervalTree instance
    Split Intervals that overlap point into two new Intervals. if
    specified, uses datafunc(interval, islower=True/False) to
    set the data field of the new Intervals.
    :param point: where to slice
    :param datafunc(interval, isupper): callable returning a new
    value for the interval's data field



In [3]:
def identify(jurisdiction):
    merged_list=tables.collections[jurisdiction]
    merged = iter(merged_list)
    accumulator = None
    continue_iteration = True
    while continue_iteration: 
        try:
            band = next(merged)
            # print (band)
            if band.has_allocation("Fixed*", primary=None):
                end_accumulation = False
                if accumulator is None:
                    accumulator = band
                else:
                    accumulator += band
            else:
                end_accumulation = True
        except StopIteration:
            end_accumulation = True
            continue_iteration = False
        except ValueError:
            end_accumulation = True
        if end_accumulation and accumulator is not None:
            # OK, we've gotten a block of primary allocations to fixed
            unaffected = True
            relevant_list = tables.find_bands(jurisdiction, accumulator.bounds, adjacent=True)
            # relevant_list = merged_list
            for other_band in relevant_list:
                relevant = other_band.overlaps(accumulator) or other_band.is_adjacent(accumulator)
                relevant = relevant and (
                    other_band.has_allocation("Radio astronomy*") or
                    other_band.has_allocation("Earth Exploration-Satellite*"))
                if relevant:
                    if unaffected:
                        print ()
                        print (f"The fixed allocations in the bands covering {accumulator.range_str()}" +
                              f" {accumulator.jurisdictions_str()} affect:")
                        #print (accumulator)
                    unaffected = False
                    print ("  " + other_band.compact_str(
                        skip_footnotes=False, 
                        skip_rules=True,
                        specific_allocations=["Earth exploration*", "Radio astronomy*"]))
            # if unaffected:
            #     print (f"============= No concern for the fixed allocations covering {accumulator.range_str()}" +
            #        f" {accumulator.jurisdictions_str()}")
            accumulator = None

In [8]:
for r in ["R1","R2","R3"]:
    print("============================================")
    identify(r)


The fixed allocations in the bands covering 13.36 MHz-13.57 MHz [R1] affect:
  13.36 MHz-13.41 MHz [R1]/RADIO ASTRONOMY/Radio astronomy 5.149#/5.149

The fixed allocations in the bands covering 25.21 MHz-25.55 MHz [R1] affect:
  25.55 MHz-25.67 MHz [R1]/RADIO ASTRONOMY/Radio astronomy 5.149#/5.149

The fixed allocations in the bands covering 29.7 MHz-47.0 MHz [R1] affect:
  37.5 MHz-38.25 MHz [R1]/Radio astronomy/Radio astronomy 5.149#/5.149

The fixed allocations in the bands covering 68.0 MHz-74.8 MHz [R1] affect:
  73.0 MHz-74.6 MHz [R1]/Radio astronomy 5.149#/

The fixed allocations in the bands covering 150.05 MHz-156.4875 MHz [R1] affect:
  150.05 MHz-153.0 MHz [R1]/RADIO ASTRONOMY/Radio astronomy 5.149#/5.149

The fixed allocations in the bands covering 223.0 MHz-328.6 MHz [R1] affect:
  322.0 MHz-328.6 MHz [R1]/RADIO ASTRONOMY/Radio astronomy 5.149#/5.149

The fixed allocations in the bands covering 401.0 MHz-406.0 MHz [R1] affect:
  401.0 MHz-402.0 MHz [R1]/EARTH EXPLORATION-